# Stage 4: Graph Mining

## Construct the graph

### import & input

In [2]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [3]:
import networkx as nx

In [4]:
import csv

In [5]:
import operator

## Preprocessing

In [4]:
f_index = 'full_paper_index.txt'
f_links = 'full_paper_reference_links.txt'
d_index = pd.read_csv(f_index, sep='\t')
d_links = pd.read_csv(f_links, sep='\t')

In [13]:
d_links['target'][1]

'NE9955'

In [19]:
d_links.loc[1][1]

'NE9955'

In [22]:
d_links.shape[0]

337394

In [238]:
def construct_mapper(d):
    mapper={}
    v=0
    for i in range(d.shape[0]):
        for j in range(2):
            if mapper.get(d.loc[i][j]) is None:
                mapper.setdefault(d.loc[i][j], v)
                v = v + 1
    print(v)
    return mapper

In [239]:
def construct_mapper_from_info(d):
    mapper={}
    v=0
    for i in range(d.shape[0]):
        if mapper.get(d.loc[i][0]) is None:
            mapper.setdefault(d.loc[i][0], v)
            v = v + 1
    print(v)
    return mapper

In [165]:
def map_writer(map, filename):
    f = open(filename, 'w')
    wr = csv.writer(f, delimiter=':', quoting=csv.QUOTE_NONE)
    for key in map.keys():
        wr.writerow( (key, map.get(key)) )
    f.close()

**Construct mappers to save time for future. DON'T run this again. **

In [242]:
paper_mapper = construct_mapper(d_links)
paper_mapper_from_info = construct_mapper_from_info(d_index)

133482
125917


In [263]:
paper_mapper = construct_mapper(d_links)

133482


In [243]:
map_writer(paper_mapper_from_info, 'paper_mapper_from_info.txt')

#### co-author graph and mapper construct

In [244]:
arr_authors = np.array(d_index['authors'])

In [245]:
authors_list=[]
for i in range(arr_authors.size):
    authors_list.append(arr_authors[i].split(" + "))

In [479]:
f = open('coauthor_graph2.txt', 'w')
wr = csv.writer(f, delimiter=':', quoting=csv.QUOTE_NONE)
for authors_of_a_paper in authors_list:
    if len(authors_of_a_paper) > 1:
        for i in range( len(authors_of_a_paper) ):
            for j in range(i+1, len(authors_of_a_paper)):
                wr.writerow( (authors_of_a_paper[i],authors_of_a_paper[j]) )
f.close()

In [247]:
all_authors_names = pd.read_csv('coauthor_graph.txt', header = None, delimiter=':', quoting=csv.QUOTE_NONE)

In [259]:
author_mapper = construct_mapper(all_authors_names)

89027


In [260]:
map_writer(author_mapper, 'author_mapper.txt')

### Why there're different number of papers? 
Seems that some source papers appeared in reference_links are not in the index file. Remove them. 

In [254]:
paper_set1=set(paper_mapper_from_info.keys())

In [255]:
paper_set2 = set(paper_mapper.keys())

In [256]:
paper_diff = paper_set2-paper_set1
paper_diff = list(paper_diff)

In [257]:
len(paper_diff)

0

In [173]:
d_links[d_links.source.isin(paper_diff) ]

,source,target
0,7DB0BE09,768A88AF
1,7DB0BE09,NE9955
2,7DB0BE09,NE9669
3,7DB0BE09,5835592B
4,7DB0BE09,NE9958
5,7DB0BE09,NE9957
6,7DB0BE09,7DE3BCB2
7,7DB0BE09,085B9585
8,7DB0BE09,NE9953
9,7DB0BE09,NE9952


In bash:
less full_paper_index.txt  | grep 7DEFE1F2 //nothing
less full_paper_reference_links.txt  | grep 7DEFE1F2 //something 

**Remove those source papers not in index.txt from reference_links map and rewrite the paper_mapper.txt file. **

In [252]:
for source in paper_diff:
    del paper_mapper[source]

In [264]:
map_writer(paper_mapper, 'paper_mapper.txt') # but this will have different number values and not continues

### Rewrite full_paper_reference_links.txt as f_paper_reference_links.txt using :

In [267]:
f = open('f_paper_reference_links.txt', 'w')
wr = csv.writer(f, delimiter=':', quoting=csv.QUOTE_NONE)
f_read = open('full_paper_reference_links.txt')
lines = f_read.readlines()
f_read.close()
for s in lines:
    s_array = s.strip().split('\t')
    if paper_mapper.get(s_array[0],-1) != -1 and paper_mapper.get(s_array[1],-1) != -1:
        wr.writerow( (paper_mapper.get(s_array[0],-1),paper_mapper.get(s_array[1],-1)) )
f.close()

### Rewrite the graph edges to numbers and reconstruct mappers

In [286]:
f = open('f_coauthor_links.txt', 'w')
wr = csv.writer(f, delimiter=':', quoting=csv.QUOTE_NONE)
f_read = open('coauthor_graph.txt')
lines = f_read.readlines()
f_read.close()
for s in lines:
    s_array = s.strip().split(':')
    if len(s_array) == 2 and (author_mapper.get(s_array[0],-1) != -1 and author_mapper.get(s_array[1],-1) != -1):
        wr.writerow( (author_mapper.get(s_array[0],-1),author_mapper.get(s_array[1],-1)) )
f.close()

In [275]:
def transform_mapper(m):
    mm={}
    for key in m.keys():
        mm.setdefault(m.get(key, -1), key)
    return mm

In [276]:
author_mapper2 = transform_mapper(author_mapper)

In [278]:
paper_mapper2 = transform_mapper(paper_mapper)

In [281]:
map_writer(paper_mapper2, 'paper_mapper2.txt')
map_writer(author_mapper2, 'author_mapper2.txt')

## Construct the graphs
- Now we only need to use author_mapper2.txt, paper_mapper2.txt as mappers. 
- f_coauthor_links.txt and f_paper_reference_links.txt as edges. 

In [301]:
G_paper = nx.DiGraph()
G_author = nx.Graph()

In [307]:
G_paper.add_nodes_from(paper_mapper2.keys())
G_author.add_nodes_from(author_mapper2.keys())

Delete last line of f_co*.txt by hand

In [309]:
E_paper = np.loadtxt('f_paper_reference_links.txt', delimiter=':')
E_author = np.loadtxt('f_coauthor_links.txt', delimiter=':')

In [310]:
G_paper.add_edges_from(E_paper)
G_author.add_edges_from(E_author)

In [315]:
def save_graph(G,sv,se):
    nodes = list(G.nodes())
    np.savetxt(sv, nodes, fmt='%d')
    edges = list(G.edges())
    np.savetxt(se, edges, fmt='%d\t%d')

In [468]:
def save_graph2(G,sv,se,mapper):# unsuccessful
    nodes = list(G.nodes())
    nodes_words=[]
    for node in nodes:
        nodes_words.append(mapper.get(node))
    np.savetxt(sv, nodes_words, fmt='%s')
    edges = list(G.edges())
    edges_words=[]
    for edge in edges:
        edges_words.append(mapper.get(edge.astype(np.int64)))
    np.savetxt(se, edges_words, fmt=['%s','%s'])

In [483]:
def save_graph3(G,sv,se):
    nodes = list(G.nodes())
    np.savetxt(sv, nodes, fmt='%s')
    edges = list(G.edges())
    np.savetxt(se, edges, fmt='%s:%s')

In [316]:
save_graph(G_paper,'p_nodes','p_edges')
save_graph(G_author,'a_nodes','a_edges')

In [484]:
G_author_words=nx.Graph()
G_author_words.add_nodes_from(author_mapper2.values())
E_author_words = np.loadtxt('coauthor_graph2.txt',delimiter=':',dtype='str')
G_author_words.add_edges_from(E_author_words)
save_graph3(G_author_words,'a_nodes_w','a_edges_w')

### Result interpreting functions

In [360]:
def paper_name_interpretation(id):
    return d_index.loc[d_index['paper_id'] == id,'title'].to_string()

In [380]:
IDs = np.array(d_index['paper_id'])
Titles = np.array(d_index['title'])

## Algos
### PageRank

In [321]:
def page_rank(G,s,mapper):
    pr = nx.pagerank(G, max_iter = 300)
    sorted_pr = sorted(pr.items(), key=operator.itemgetter(1), reverse=True)
    sorted_pr_byname = [(mapper[index], rank) for (index, rank) in sorted_pr]
    f_pr = open(s,'w')
    wr = csv.writer(f_pr, quoting=csv.QUOTE_NONE)
    for line in sorted_pr_byname:
        wr.writerow(line)
    f_pr.close()

In [322]:
page_rank(G_author, 'pagerank_a', author_mapper2)# jiawei han toka appeared

In [323]:
page_rank(G_paper, 'pagerank_p', paper_mapper2)

In [43]:
f_read = open('pagerank_a')
lines = f_read.readlines()
f_read.close()
ranked_paper_titles=[]
for s in lines:
    s_array = s.strip().split(',')
    ranked_paper_titles.append(Titles[np.where(IDs==s_array[0])][0])

NameError: name 'Titles' is not defined

In [44]:
ranked_paper_titles

[]

### Community detection

In [420]:
from sklearn import cross_validation
from networkx.algorithms import community
import cPickle as cpk

In [418]:
component_list_author = sorted(nx.connected_component_subgraphs(G_author), key=len, reverse = True)
component_size_author = [subG.number_of_nodes() for subG in component_list_author]
np.savetxt('component_size_author.txt',component_size_author, fmt='%d')
Gmax_author = component_list_author[0] #7268 components

In [419]:
G_paper_undirected = G_paper.to_undirected()
component_list_paper = sorted(nx.connected_component_subgraphs(G_paper_undirected), key=len, reverse = True)
component_size_paper = [subG.number_of_nodes() for subG in component_list_paper]
np.savetxt('component_size_paper.txt',component_size_paper, fmt='%d')
Gmax_paper = component_list_paper[0] #397 components

In [485]:
component_list_author_words = sorted(nx.connected_component_subgraphs(G_author_words), key=len, reverse = True)
component_size_author_words = [subG.number_of_nodes() for subG in component_list_author_words]
np.savetxt('component_size_author_words.txt',component_size_author_words, fmt='%d')
Gmax_author_words = component_list_author_words[0] 

In [425]:
save_graph(Gmax_author,'Gmax_author_nodes','Gmax_author_edges')

### Sample Gmax_author and save it

In [486]:
from random import choice
random_nodes=[]
for i in range(6000):
    random_node = choice(Gmax_author_words.nodes())
    random_nodes.append(random_node)
subGmax_author_words = Gmax_author_words.subgraph(random_nodes)

In [487]:
save_graph3(subGmax_author_words,'subGmax_author_nodes_words.csv','subGmax_author_edges_words.csv')

### From now on we will use Gmax's to replace G's, and use only authors
### k-clique

In [421]:
def kclique(G,mapper):
    k_clique_comm = sorted(list(nx.k_clique_communities(G,15)), key=len, reverse = True)
    num_cliques = len(k_clique_comm)
    max_k_clique_comm = [[mapper[index] for index in list(k_clique_comm[i])] for i in range(num_cliques)]
    for i in range(num_cliques):
        f = open('kcliques/clique'+str(i)+'.txt','w')
        cpk.dump(max_k_clique_comm[i],f)
        f.close()

In [492]:
def kclique2(G):
    k_clique_comm = sorted(list(nx.k_clique_communities(G,20)), key=len, reverse = True)
    num_cliques = len(k_clique_comm)
    max_k_clique_comm = [[index for index in list(k_clique_comm[i])] for i in range(num_cliques)]
    for i in range(num_cliques):
        f = open('subkcliques/clique'+str(i)+'.txt','w')
        cpk.dump(max_k_clique_comm[i],f)
        f.close()

In [423]:
kclique(Gmax_author,author_mapper2)

In [493]:
kclique2(subGmax_author_words)

### link prediction
use jaccard

In [28]:
from random import choice
random_nodes2=[]
for i in range(3000):#50000/63364
    random_node = choice(Gmax_author_words.nodes())
    random_nodes2.append(random_node)

random_nodes3=[]
for i in range(2000):#50000/63364
    random_node = choice(random_nodes2)
    random_nodes3.append(random_node)    
    
all_nodes_set = set(random_nodes2)
train_nodes_set = set(random_nodes3)
test_nodes_set = all_nodes_set - train_nodes_set

train_Gmax_author_words = Gmax_author_words.subgraph(random_nodes2)
test_Gmax_author_words = Gmax_author_words.subgraph(test_nodes_set)

In [36]:
all_edges_set = set(Gmax_author_words.edges())
test_edges_set = set(test_Gmax_author_words.edges())
train_edges_set = all_edges_set - test_edges_set

train_Gmax_author_words2=nx.Graph()
train_Gmax_author_words2.add_nodes_from(Gmax_author_words.nodes())
train_Gmax_author_words2.add_edges_from(train_edges_set)

In [37]:
preds = nx.jaccard_coefficient(train_Gmax_author_words2)

In [40]:
f = open('link_preds.csv', 'w')
wr=csv.writer(f, quoting=csv.QUOTE_NONE,delimiter=':')
i=0
for u,v,p in preds:
    if Gmax_author_words.has_edge(u,v):
        i=i+p
    else:
        i=i-p
        #'(%s, %s) -> %.8f' % (u, v, p)  
f.close()

### Use smaller subgraph

In [10]:
Gmax_author_words = nx.Graph()
V=np.loadtxt('subGmax_author_nodes_words.csv',delimiter=':',dtype='str')
Gmax_author_words.add_nodes_from(V)
E = np.loadtxt('subGmax_author_edges_words.csv',delimiter=':',dtype='str')
Gmax_author_words.add_edges_from(E)
print Gmax_author_words.number_of_edges()

4068


Compute a percentage score

In [42]:
Gmax_author_words.number_of_nodes()

5716

In [41]:
i

-957.6274413832608